
**Install requirements**

In [0]:
!pip3 install 'torch==1.3.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

     |████████████████████████████████| 773.1MB 23kB/s 
ERROR: torchvision 0.6.0+cu101 has requirement torch==1.5.0, but you'll have torch 1.3.0 which is incompatible.
  Found existing installation: torch 1.5.0+cu101
    Uninstalling torch-1.5.0+cu101:
      Successfully uninstalled torch-1.5.0+cu101
     |████████████████████████████████| 4.0MB 2.8MB/s 
     |████████████████████████████████| 753.4MB 19kB/s 
  Found existing installation: torch 1.3.0
    Uninstalling torch-1.3.0:
      Successfully uninstalled torch-1.3.0
  Found existing installation: torchvision 0.6.0+cu101
    Uninstalling torchvision-0.6.0+cu101:
      Successfully uninstalled torchvision-0.6.0+cu101
     |████████████████████████████████| 634kB 2.8MB/s 
  Created wheel for Pillow-SIMD: filename=Pillow_SIMD-7.0.0.post3-cp36-cp36m-linux_x86_64.whl size=1110246 sha256=8160f6f3236f427ec3e2ff95573b3b288346ecd1c7f7628b08c27bb1965cc63a
  Stored in directory: /root/.cache/pip/wheels/d3/ac/4f/4cdf8febba528e5f1b09fc58d5181

**Import libraries**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torchvision.models import alexnet
from torch.autograd import Function 
try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV

#**Set Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

#vectors for grid search
Learning_rates = [1e-5, 1e-4,1e-2]


num_epochs = [10, 20, 30]

LOG_FREQUENCY = 10

# **Define Data Preprocessing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                     
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))                                    
])

#**Prepare Dataset**

In [0]:
# Clone github repository with data
if not os.path.isdir('./Homework3-PACS'):
  !git clone https://github.com/ele9996/Homework3-PACS.git

DATA_DIR = 'Homework3-PACS/PACS'

# Prepare Pytorch train/test Datasets
train_dataset = torchvision.datasets.ImageFolder(DATA_DIR + "/photo", transform=train_transform)
test_dataset = torchvision.datasets.ImageFolder(DATA_DIR + "/art_painting", transform=eval_transform)

#from old template
#train_indexes = [idx for idx in range(len(train_dataset)) if idx % 5]
#test_indexes = [idx for idx in range(len(test_dataset)) if not idx % 5]

#train_dataset = Subset(train_dataset, train_indexes)
#test_dataset = Subset(test_dataset, test_indexes)

# Check dataset sizes
print('Train Dataset: {}'.format(len(train_dataset)))
print('Test Dataset: {}'.format(len(test_dataset)))

Cloning into 'Homework3-PACS'...
remote: Enumerating objects: 10032, done.
remote: Total 10032 (delta 0), reused 0 (delta 0), pack-reused 10032
Receiving objects: 100% (10032/10032), 174.13 MiB | 11.47 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Checking out files: 100% (9993/9993), done.
Train Dataset: 1670
Test Dataset: 2048


#**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
#photos
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True) #num_workers=4
#art painting
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

#**Prepare DACNN and main function definition**

In [0]:
model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}



class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        # Store context for backprop
        ctx.alpha = alpha
        
        # Forward pass is a no-op
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        # Backward pass is just to -alpha the gradient
        output = grad_output.neg() * ctx.alpha
        return output, None




class DACNN(nn.Module):
  def __init__(self, num_classes, **kwargs):
        super(DACNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
            nn.Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False),
           
          )
        self.avgpool= nn.AdaptiveAvgPool2d(output_size=(6, 6))
        
        self.classifier = nn.Sequential(
            
            nn.Dropout(),
            nn.Linear(in_features=9216, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=4096, out_features=1000, bias=True),
            
          )
        
        self.domain_classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096), 
            nn.BatchNorm1d(4096),
            nn.ReLU(True),
            nn.Linear(4096, 1000),
            nn.LogSoftmax(dim=1),
          )
          
  def forward(self, x, alpha=None):     
        # Handle single-channel input by expanding (repeating) the singleton dimention
        features=self.features(x)
        features = features.view(features.size(0), -1)

        if alpha is not None:
            # gradient reversal layer (backward gradients will be reversed)
            reverse_feature = GradientReversalFn.apply(features, alpha)
            domain_pred = self.domain_classifier(features)
            return domain_pred
        # If we don't pass alpha, we assume we are training with supervision
        else:
            
            class_pred = self.classifier(features)
            return class_pred
        
        
       

def DANNAlexnet(pretrained=True,  num_classes=7, **kwargs):
    net = DACNN(num_classes = 7, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'])
        net.load_state_dict(state_dict, strict=False)
        for x in [(0,1) , (3,6)]:
            
            net.domain_classifier[x[0]].weight.data = net.classifier[x[1]].weight.data
            net.domain_classifier[x[0]].bias.data = net.classifier[x[1]].bias.data
        net.classifier[6] = nn.Linear(4096, num_classes)
        net.domain_classifier[3] = nn.Linear(4096, 2)
    return net

net=DANNAlexnet(num_classes=7)
net

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


DACNN(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
   

#**Prepare Training**

In [0]:
# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

#Training without adaptation

In [0]:
net=net.to(DEVICE)
cudnn.benchmark

False

In [0]:


actual_step=0;
num_corrects=0

for epoch in range(NUM_EPOCHS):
  scheduler.step() 
  
  for images, labels in  tqdm(train_dataloader):

    
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() 

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

  
    outputs = net(images)

    loss = criterion(outputs, labels)

    # Log loss
    if actual_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(actual_step, loss.item()))

  

    loss.backward() 

    optimizer.step() 

    actual_step += 1
    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    num_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = num_corrects / float(len(train_dataset))
  print("Accuracy on training set = "  + str(accuracy))
  num_corrects = 0

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/6 [00:00<?, ?it/s]

Step 0, Loss 2.4324557781219482


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.3502994011976048


 67%|██████▋   | 4/6 [00:06<00:03,  1.92s/it]

...

Step 170, Loss 0.02422325685620308


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.9083832335329342


100%|██████████| 6/6 [00:07<00:00,  1.27s/it]

Accuracy on training set = 0.9143712574850299


#**Test**

In [0]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 8/8 [00:08<00:00,  1.11s/it]

Test Accuracy: 0.47021484375


#Train With Domain Adaptation

In [0]:
alpha=1

net = DANNAlexnet(num_classes = 7)
net = net.to(DEVICE)

class_loss = nn.CrossEntropyLoss() 
domain_loss = nn.CrossEntropyLoss()

parameters_to_optimize = net.parameters() 

optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


In [0]:
max_batches = max(len(train_dataloader), len(test_dataloader))
min_batches = min(len(train_dataloader), len(test_dataloader))#
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark #function to optimize runtime by finding the best configuration for your hw, useful when input size 
running_corrects = 0
current_step = 0
for epoch in range(NUM_EPOCHS):
  print(f'\nEpoch {epoch+1:04d} / {NUM_EPOCHS:04d}', end='\n=================\n')
  scheduler.step() 
  train_iterable = iter(train_dataloader)
  test_iterable = iter(test_dataloader)
  for batch in range(max_batches):
    net.train() # Sets module in training mode
    optimizer.zero_grad() # Zero-ing the gradients 
    if( batch == min_batches):#
      train_iterable = iter(train_dataloader)#

    images_source, labels_source = next(train_iterable)
    labels_domain = torch.zeros(len(images_source), dtype=torch.long)    
    
    images_source = images_source.to(DEVICE)
    labels_source = labels_source.to(DEVICE)
    labels_domain = labels_domain.to(DEVICE)
    
    class_output = net(images_source)
    domain_output = net(images_source, alpha)
    


    loss_s_label = class_loss(class_output, labels_source)
    loss_s_domain = domain_loss(domain_output, labels_domain)
    

  
    targets, _ = next(test_iterable)
    target_domain = torch.ones(len(targets), dtype=torch.long)

 
    targets = targets.to(DEVICE)
    target_domain = target_domain.to(DEVICE)

    target_output = net(targets, alpha)


    loss_t_domain = domain_loss(target_output,target_domain)


    loss = loss_s_label + loss_s_domain + loss_t_domain
    loss.backward() 

    optimizer.step() 

    current_step += 1

    print(f'[{batch+1}/{max_batches}] '
          f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain.item():.4f} '
          f't_domain_loss: {loss_t_domain.item():.4f} '
          )  






Epoch 0001 / 0030


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1/8] class_loss: 1.9753 s_domain_loss: 0.7616 t_domain_loss: 0.6761 
[2/8] class_loss: 1.7905 s_domain_loss: 0.7219 t_domain_loss: 0.7006 
[3/8] class_loss: 1.5045 s_domain_loss: 0.6889 t_domain_loss: 0.7369 
[4/8] class_loss: 1.2210 s_domain_loss: 0.6622 t_domain_loss: 0.7565 
[5/8] class_loss: 1.0798 s_domain_loss: 0.6637 t_domain_loss: 0.7511 
[6/8] class_loss: 1.0463 s_domain_loss: 0.6879 t_domain_loss: 0.7150 
[7/8] class_loss: 0.7232 s_domain_loss: 0.7178 t_domain_loss: 0.6848 
[8/8] class_loss: 0.6229 s_domain_loss: 0.7310 t_domain_loss: 0.6680 

Epoch 0002 / 0030
[1/8] class_loss: 0.5683 s_domain_loss: 0.7125 t_domain_loss: 0.6796 
[2/8] class_loss: 0.5145 s_domain_loss: 0.6891 t_domain_loss: 0.7012 
[3/8] class_loss: 0.3797 s_domain_loss: 0.6582 t_domain_loss: 0.7198 
[4/8] class_loss: 0.3414 s_domain_loss: 0.6552 t_domain_loss: 0.7267 
[5/8] class_loss: 0.3154 s_domain_loss: 0.6593 t_domain_loss: 0.7171 
[6/8] class_loss: 0.3035 s_domain_loss: 0.6844 t_domain_loss: 0.6954 
[

#Test

In [0]:

net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  loss = criterion(outputs, labels)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))
print('Loss is: ' + str(loss.item()))

100%|██████████| 8/8 [00:09<00:00,  1.17s/it]

Test Accuracy: 0.4580078125
Loss is: 8.91762638092041


#(EXTRA) Cross Domain Validation

##Preparing new dataset

In [0]:

# first validation set --> Cartoon
cartoon_dataset = torchvision.datasets.ImageFolder(DATA_DIR + "/cartoon", transform=eval_transform)

# second validation set --> Sketch
sketch_dataset = torchvision.datasets.ImageFolder(DATA_DIR + "/sketch", transform=eval_transform)


print('Cartoon Dataset: {}'.format(len(cartoon_dataset)))
print('Sketch Dataset: {}'.format(len(sketch_dataset)))


Cartoon Dataset: 2344
Sketch Dataset: 3929


##Prepare Dataloader

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
#photos
cartoon_dataloader = DataLoader(cartoon_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4) 
# sketch
sketch_dataloader = DataLoader(sketch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

##Grid search on on photo to cartoon and sketch without domain adaptation

###Validation on cartoon

In [0]:
def valOnCartoon():
    #net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
    net.train(False) # Set Network to evaluation mode

    running_corrects = 0
    for images, labels in tqdm(cartoon_dataloader):
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      # Forward Pass
      outputs = net(images)

      # Get predictions
      _, preds = torch.max(outputs.data, 1)

      # Update Corrects
      running_corrects += torch.sum(preds == labels.data).data.item()

    # Calculate Accuracy
    accuracy = running_corrects / float(len(test_dataset))
    #Calculate Loss
    loss = criterion(outputs, labels)

    print('Validation Accuracy on Cartoon: {}'.format(accuracy))
    return (accuracy, loss.item())

In [0]:
max_val_accuracy=0
min_val_loss=100
best_LR=0
best_epoch_num=0
for lrate in Learning_rates:
  
  for epochs in num_epochs:
      print("hyperparameters are: LR={} and NUM_EPOCHS = {}".format(lrate,epochs))
      
      # Define loss function
      criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

      # Choose parameters to optimize
      # To access a different set of parameters, you have to access submodules of AlexNet
      # (nn.Module objects, like AlexNet, implement the Composite Pattern)
      # e.g.: parameters of the fully connected layers: net.classifier.parameters()
      # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
      parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

      # Define optimizer
      # An optimizer updates the weights based on loss
      # We use SGD with momentum
      optimizer = optim.SGD(parameters_to_optimize, lr=lrate, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

      # Define scheduler
      # A scheduler dynamically changes learning rate
      # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
      net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
      cudnn.benchmark
      
      actual_step=0;
      num_corrects=0



      for epoch in range(epochs):
        scheduler.step() 
        
        for images, labels in  tqdm(train_dataloader):

          
          images = images.to(DEVICE)
          labels = labels.to(DEVICE)

          net.train() 

          # PyTorch, by default, accumulates gradients after each backward pass
          # We need to manually set the gradients to zero before starting a new iteration
          optimizer.zero_grad() # Zero-ing the gradients

        
          outputs = net(images)

          loss = criterion(outputs, labels)

          # Log loss
          if actual_step % LOG_FREQUENCY == 0:
            print('Step {}, Loss {}'.format(actual_step, loss.item()))

        

          loss.backward() 

          optimizer.step() 

          actual_step += 1
          # Get predictions
          _, preds = torch.max(outputs.data, 1)

          # Update Corrects
          num_corrects += torch.sum(preds == labels.data).data.item()

        # Calculate Accuracy
        accuracy = num_corrects / float(len(train_dataset))

        print("Accuracy on training set = "  + str(accuracy))

        

        
        num_corrects = 0

      
      params= valOnCartoon()
      val_accuracy=params[0]
      val_loss=params[1]
      if val_loss<min_val_loss:
        max_val_accuracy=val_accuracy
        min_val_loss=val_loss
        
        best_LR= lrate
        best_epoch_num=epochs
        
      
      if val_loss==min_val_loss:
        if val_accuracy>max_val_accuracy:
          max_val_accuracy=val_accuracy
          min_val_loss=val_loss
          
          best_LR= lrate
          best_epoch_num=epochs  
            


print ("best hyperparemeters for validation on cartoon are : LR={}, NUM_EPOCHS= {} having highest validation accuracy = {} and minimum validation loss ={}".format(best_LR, best_epoch_num,max_val_accuracy,min_val_loss))



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/6 [00:00<?, ?it/s]

hyperparameters are: LR=1e-05 and NUM_EPOCHS = 10
Step 0, Loss 0.04220133274793625


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.9125748502994012


 67%|██████▋   | 4/6 [00:05<00:03,  1.81s/it]

...

Accuracy on training set = 0.9197604790419162


  0%|          | 0/10 [00:00<?, ?it/s]

Accuracy on training set = 0.9197604790419162


100%|██████████| 10/10 [00:09<00:00,  1.04it/s]

Validation Accuracy on Cartoon: 0.28271484375
best hyperparemeters for validation on cartoon are : LR=1e-05, NUM_EPOCHS= 10 having highest validation accuracy = 0.2685546875 and minimum validation loss =7.355759620666504


###Validation on Sketch

In [0]:
def valOnSketch():
   # net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
    net.train(False) # Set Network to evaluation mode

    running_corrects = 0
    for images, labels in tqdm(sketch_dataloader):
      images = images.to(DEVICE)
      labels = labels.to(DEVICE)

      # Forward Pass
      outputs = net(images)

      # Get predictions
      _, preds = torch.max(outputs.data, 1)

      # Update Corrects
      running_corrects += torch.sum(preds == labels.data).data.item()

    # Calculate Accuracy
    accuracy = running_corrects / float(len(test_dataset))
    #Calculate Loss
    loss = criterion(outputs, labels)

    print('Validation Accuracy on Sketch: {}'.format(accuracy))
    return (accuracy, loss.item())

In [0]:
max_val_accuracy2=0
min_val_loss2=100
best_LR2=0
best_epoch_num2=0
for lrate in Learning_rates:
  
  for epochs in num_epochs:
      print("hyperparameters are: LR={} and NUM_EPOCHS = {}".format(lrate,epochs))
      
      # Define loss function
      criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

      # Choose parameters to optimize
      # To access a different set of parameters, you have to access submodules of AlexNet
      # (nn.Module objects, like AlexNet, implement the Composite Pattern)
      # e.g.: parameters of the fully connected layers: net.classifier.parameters()
      # e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
      parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

      # Define optimizer
      # An optimizer updates the weights based on loss
      # We use SGD with momentum
      optimizer = optim.SGD(parameters_to_optimize, lr=lrate, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

      # Define scheduler
      # A scheduler dynamically changes learning rate
      # The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)
      net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
      cudnn.benchmark
      
      actual_step=0;
      num_corrects=0



      for epoch in range(epochs):
        scheduler.step() 
        
        for images, labels in  tqdm(train_dataloader):

          
          images = images.to(DEVICE)
          labels = labels.to(DEVICE)

          net.train() 

          # PyTorch, by default, accumulates gradients after each backward pass
          # We need to manually set the gradients to zero before starting a new iteration
          optimizer.zero_grad() # Zero-ing the gradients

        
          outputs = net(images)

          loss = criterion(outputs, labels)

          # Log loss
          if actual_step % LOG_FREQUENCY == 0:
            print('Step {}, Loss {}'.format(actual_step, loss.item()))

        

          loss.backward() 

          optimizer.step() 

          actual_step += 1
          # Get predictions
          _, preds = torch.max(outputs.data, 1)

          # Update Corrects
          num_corrects += torch.sum(preds == labels.data).data.item()

        # Calculate Accuracy
        accuracy = num_corrects / float(len(train_dataset))

        print("Accuracy on training set = "  + str(accuracy))

        

        
        num_corrects = 0

      
      params= valOnSketch()
      val_accuracy=params[0]
      val_loss=params[1]
      if val_loss<min_val_loss2:
        max_val_accuracy2=val_accuracy
        min_val_loss2=val_loss
        
        best_LR2= lrate
        best_epoch_num2=epochs
        
      
      if val_loss==min_val_loss2:
        if val_accuracy>max_val_accuracy2:
          max_val_accuracy2=val_accuracy
          min_val_loss2=val_loss
          
          best_LR2= lrate
          best_epoch_num2=epochs  
            


print ("best hyperparemeters for validation on sketch are : LR={}, NUM_EPOCHS= {} having highest validation accuracy = {} and minimum validation loss ={}".format(best_LR2, best_epoch_num2,max_val_accuracy2,min_val_loss2))




/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/6 [00:00<?, ?it/s]

hyperparameters are: LR=1e-05 and NUM_EPOCHS = 10
Step 0, Loss 9.295716881752014e-05


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.9197604790419162


 67%|██████▋   | 4/6 [00:05<00:03,  1.87s/it]

...

Accuracy on training set = 0.9197604790419162


  0%|          | 0/16 [00:00<?, ?it/s]

Accuracy on training set = 0.9197604790419162


100%|██████████| 16/16 [00:14<00:00,  1.08it/s]

Validation Accuracy on Sketch: 0.5439453125
best hyperparemeters for validation on sketch are : LR=0.0001, NUM_EPOCHS= 30 having highest validation accuracy = 0.5615234375 and minimum validation loss =10.133872032165527


##Evaluate the best hyperparameters among the two validation sets

In [0]:
#I take the best hyperparameters belonging to the validation set where the loss is smaller

if min_val_loss<min_val_loss2:
  best_final_LR= best_LR
  best_final_epoch_num= best_epoch_num
else:
    best_final_LR= best_LR2
    best_final_epoch_num= best_epoch_num2

print("best final LR= {}, best final epoch number={}".format(best_final_LR,best_final_epoch_num))

best final LR= 1e-05, best final epoch number=10


##Train on Photo and Test on Art painting with the best hyperparameters without domain adaptation

###Train on Photo

In [0]:

# Define loss function
criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy

# Choose parameters to optimize
# To access a different set of parameters, you have to access submodules of AlexNet
# (nn.Module objects, like AlexNet, implement the Composite Pattern)
# e.g.: parameters of the fully connected layers: net.classifier.parameters()
# e.g.: parameters of the convolutional layers: look at alexnet's source code ;) 
parameters_to_optimize = net.parameters() # In this case we optimize over all the parameters of AlexNet

# Define optimizer
# An optimizer updates the weights based on loss
# We use SGD with momentum
optimizer = optim.SGD(parameters_to_optimize, lr=best_final_LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

# Define scheduler
# A scheduler dynamically changes learning rate
# The most common schedule is the step(-down), which multiplies learning rate by gamma every STEP_SIZE epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)



net=net.to(DEVICE)
cudnn.benchmark

actual_step=0;
num_corrects=0

for epoch in range(best_final_epoch_num):
  scheduler.step() 
  
  for images, labels in  tqdm(train_dataloader):

    
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() 

    # PyTorch, by default, accumulates gradients after each backward pass
    # We need to manually set the gradients to zero before starting a new iteration
    optimizer.zero_grad() # Zero-ing the gradients

  
    outputs = net(images)

    loss = criterion(outputs, labels)

    # Log loss
    if actual_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(actual_step, loss.item()))

  

    loss.backward() 

    optimizer.step() 

    actual_step += 1
    # Get predictions
    _, preds = torch.max(outputs.data, 1)

    # Update Corrects
    num_corrects += torch.sum(preds == labels.data).data.item()

  # Calculate Accuracy
  accuracy = num_corrects / float(len(train_dataset))
  print("Accuracy on training set = "  + str(accuracy))
  num_corrects = 0


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
  0%|          | 0/6 [00:00<?, ?it/s]

Step 0, Loss 0.0003589130938053131


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.9197604790419162


 67%|██████▋   | 4/6 [00:06<00:03,  1.87s/it]

...

Step 50, Loss 0.00012804754078388214


  0%|          | 0/6 [00:00<?, ?it/s]

Accuracy on training set = 0.9197604790419162


100%|██████████| 6/6 [00:07<00:00,  1.23s/it]

Accuracy on training set = 0.9197604790419162


###Test on Art Painting 

In [0]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))

100%|██████████| 8/8 [00:08<00:00,  1.08s/it]

Test Accuracy: 0.4677734375


##Grid search on photo to cartoon and sketch with domain adaptation


###Validation on Cartoon

In [0]:
#ho provato a mettere 1e-2 (come prima) ma era troppo alto
#il test set va bene con i lr alti mentre i due validation vanno bene con quelli piccoli(con quelli grandi divergono), provo quindi a cercare i più grandi tra i parametri possibili che non facciano divergere il modello e in contemporanea massimizzino il rate
#Ho visto che 9e-2 ha una test accuracy troppo basso per cui l'ordine di grandezza sarà -3...
#Learning_rates = [1e-5, 1e-4,1e-2]
Learning_rates = [5e-3,6e-3, 7e-3]

In [0]:
max_val_accuracy=0
min_val_loss=100
best_LR=0
best_epoch_num=0
for lrate in Learning_rates:
   for epochs in num_epochs:

      print("hyperprameters are: LR={} and NUM_EPOCHS = {}".format(lrate, epochs))
      alpha= 1
      net = DANNAlexnet(num_classes = 7)
      net = net.to(DEVICE)

      class_loss = nn.CrossEntropyLoss() 
      domain_loss = nn.CrossEntropyLoss()

      parameters_to_optimize = net.parameters() 

      optimizer = optim.SGD(parameters_to_optimize, lr=lrate, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


      max_batches = max(len(train_dataloader), len(cartoon_dataloader))
      min_batches = min(len(train_dataloader), len(cartoon_dataloader))#
      net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

      cudnn.benchmark #function to optimize runtime by finding the best configuration for your hw, useful when input size 
      running_corrects = 0
      current_step = 0
      for epoch in range(epochs):
        print(f'\nEpoch {epoch+1:04d} / {epochs:04d}', end='\n=================\n')
        scheduler.step() 
        train_iterable = iter(train_dataloader)
        test_iterable = iter(cartoon_dataloader)
        for batch in range(max_batches):
          net.train() # Sets module in training mode
          optimizer.zero_grad() # Zero-ing the gradients 
          if( batch == min_batches):#
            train_iterable = iter(train_dataloader)#

          images_source, labels_source = next(train_iterable)
          labels_domain = torch.zeros(len(images_source), dtype=torch.long)    
          
          images_source = images_source.to(DEVICE)
          labels_source = labels_source.to(DEVICE)
          labels_domain = labels_domain.to(DEVICE)
          
          class_output = net(images_source)
          domain_output = net(images_source, alpha)
          


          loss_s_label = class_loss(class_output, labels_source)
          loss_s_domain = domain_loss(domain_output, labels_domain)
          

        
          targets, _ = next(test_iterable)
          target_domain = torch.ones(len(targets), dtype=torch.long)

      
          targets = targets.to(DEVICE)
          target_domain = target_domain.to(DEVICE)

          target_output = net(targets, alpha)


          loss_t_domain = domain_loss(target_output,target_domain)


          loss = loss_s_label + loss_s_domain + loss_t_domain
          loss.backward() 

          optimizer.step() 

          current_step += 1

          print(f'[{batch+1}/{max_batches}] '
                f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain.item():.4f} '
                f't_domain_loss: {loss_t_domain.item():.4f} '
                )  
        

      params=valOnCartoon()
      val_accuracy= params[0]
      val_loss=params[1]
      
      
      if val_loss<min_val_loss:
        max_val_accuracy=val_accuracy
        min_val_loss=val_loss
        
        best_LR= lrate
        best_epoch_num= epochs
        
      
      if val_loss<min_val_loss:
        if val_accuracy>max_val_accuracy:
          max_val_accuracy=val_accuracy
          min_val_loss=val_loss
          
          best_LR= lrate
          best_epoch_num=epochs
      
      
      
            


print ("best hyperparemeters after validation on cartoon are : LR={}, NUM_EPOCHS= {} having validation accuracy= {} and validation loss ={}".format(best_LR, best_epoch_num,max_val_accuracy,min_val_loss))     





hyperprameters are: LR=0.005 and NUM_EPOCHS = 10

Epoch 0001 / 0010


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1/10] class_loss: 2.1891 s_domain_loss: 0.5277 t_domain_loss: 0.9407 
[2/10] class_loss: 1.3782 s_domain_loss: 1.2009 t_domain_loss: 0.3842 
[3/10] class_loss: 0.8773 s_domain_loss: 0.4380 t_domain_loss: 1.0669 
[4/10] class_loss: 0.5688 s_domain_loss: 0.7007 t_domain_loss: 0.7003 
[5/10] class_loss: 0.4552 s_domain_loss: 0.9893 t_domain_loss: 0.4784 
[6/10] class_loss: 0.2760 s_domain_loss: 0.4092 t_domain_loss: 1.0975 
[7/10] class_loss: 0.2194 s_domain_loss: 0.8722 t_domain_loss: 0.5253 
[8/10] class_loss: 0.2300 s_domain_loss: 0.7537 t_domain_loss: 0.6200 
[9/10] class_loss: 0.1697 s_domain_loss: 0.4430 t_domain_loss: 0.9877 
[10/10] class_loss: 0.1743 s_domain_loss: 0.9731 t_domain_loss: 0.4710 

Epoch 0002 / 0010
[1/10] class_loss: 0.2037 s_domain_loss: 0.6036 t_domain_loss: 0.7471 
[2/10] class_loss: 0.1456 s_domain_loss: 0.5067 t_domain_loss: 0.8401 
[3/10] class_loss: 0.1058 s_domain_loss: 0.9711 t_domain_loss: 0.4170 
[4/10] class_loss: 0.2269 s_domain_loss: 0.4576 t_domain_

  0%|          | 0/10 [00:00<?, ?it/s]

[10/10] class_loss: 0.1297 s_domain_loss: 0.1542 t_domain_loss: 0.4333 


...

Validation Accuracy on Cartoon: 0.2275390625
hyperprameters are: LR=0.007 and NUM_EPOCHS = 30

Epoch 0001 / 0030
[1/10] class_loss: 2.0417 s_domain_loss: 0.5381 t_domain_loss: 0.9218 
[2/10] class_loss: 1.2032 s_domain_loss: 1.5071 t_domain_loss: 0.2722 
[3/10] class_loss: 0.7454 s_domain_loss: 0.1023 t_domain_loss: 2.3800 
[4/10] class_loss: 0.4497 s_domain_loss: 1.9988 t_domain_loss: 0.1586 
[5/10] class_loss: 0.3450 s_domain_loss: 0.3021 t_domain_loss: 1.3382 
[6/10] class_loss: 0.4050 s_domain_loss: 1.0312 t_domain_loss: 0.4462 
[7/10] class_loss: 0.2421 s_domain_loss: 0.4211 t_domain_loss: 1.0476 
[8/10] class_loss: 0.3472 s_domain_loss: 1.3471 t_domain_loss: 0.3010 
[9/10] class_loss: 0.2236 s_domain_loss: 0.1479 t_domain_loss: 1.9491 
[10/10] class_loss: 0.1660 s_domain_loss: 2.0668 t_domain_loss: 0.1332 

Epoch 0002 / 0030
[1/10] class_loss: 0.1400 s_domain_loss: 0.2082 t_domain_loss: 1.5636 
[2/10] class_loss: 0.1840 s_domain_loss: 1.1871 t_domain_loss: 0.3239 
[3/10] class_lo

  0%|          | 0/10 [00:00<?, ?it/s]

[10/10] class_loss: nan s_domain_loss: nan t_domain_loss: nan 


100%|██████████| 10/10 [00:09<00:00,  1.00it/s]

Validation Accuracy on Cartoon: 0.18994140625
best hyperparemeters after validation on cartoon are : LR=0.007, NUM_EPOCHS= 20 having validation accuracy= 0.2275390625 and validation loss =1.7302061319351196


###Validation on Sketch

In [0]:
max_val_accuracy2=0
min_val_loss2=100
best_LR2=0
best_epoch_num2=0
for lrate in Learning_rates:
   for epochs in num_epochs:

      print("hyperprameters are: LR={} and NUM_EPOCHS = {}".format(lrate, epochs))
      alpha= 1
      net = DANNAlexnet(num_classes = 7)
      net = net.to(DEVICE)

      class_loss = nn.CrossEntropyLoss() 
      domain_loss = nn.CrossEntropyLoss()

      parameters_to_optimize = net.parameters() 

      optimizer = optim.SGD(parameters_to_optimize, lr=lrate, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

      scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


      max_batches = max(len(train_dataloader), len(sketch_dataloader))
      min_batches = min(len(train_dataloader), len(sketch_dataloader))#
      net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

      cudnn.benchmark #function to optimize runtime by finding the best configuration for your hw, useful when input size 
      running_corrects = 0
      current_step = 0
      for epoch in range(epochs):
        print(f'\nEpoch {epoch+1:04d} / {epochs:04d}', end='\n=================\n')
        scheduler.step() 
        train_iterable = iter(train_dataloader)
        test_iterable = iter(sketch_dataloader)
        for batch in range(max_batches):
          net.train() # Sets module in training mode
          optimizer.zero_grad() # Zero-ing the gradients 
          
            
          try:
            image_source, labels_source = next(train_iterable)
          except StopIteration:
            train_iterable = iter(train_dataloader)
            image_source, labels_source = next(train_iterable)

          images_source, labels_source = next(train_iterable)
          labels_domain = torch.zeros(len(images_source), dtype=torch.long)    
          
          images_source = images_source.to(DEVICE)
          labels_source = labels_source.to(DEVICE)
          labels_domain = labels_domain.to(DEVICE)
          
          class_output = net(images_source)
          domain_output = net(images_source, alpha)
          


          loss_s_label = class_loss(class_output, labels_source)
          loss_s_domain = domain_loss(domain_output, labels_domain)
          

        
          targets, _ = next(test_iterable)
          target_domain = torch.ones(len(targets), dtype=torch.long)

      
          targets = targets.to(DEVICE)
          target_domain = target_domain.to(DEVICE)

          target_output = net(targets, alpha)


          loss_t_domain = domain_loss(target_output,target_domain)


          loss = loss_s_label + loss_s_domain + loss_t_domain
          loss.backward() 

          optimizer.step() 

          current_step += 1

          print(f'[{batch+1}/{max_batches}] '
                f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain.item():.4f} '
                f't_domain_loss: {loss_t_domain.item():.4f} '
                )  
        

      params=valOnSketch()
      val_accuracy= params[0]
      val_loss=params[1]
      
      
      if val_loss<min_val_loss2:
        max_val_accuracy2=val_accuracy
        min_val_loss2=val_loss
        
        best_LR2= lrate
        best_epoch_num2= epochs
        
      
      if val_loss<min_val_loss2:
        if val_accuracy>max_val_accuracy2:
          max_val_accuracy2=val_accuracy
          min_val_loss2=val_loss
          
          best_LR2= lrate
          best_epoch_num2=epochs
      
      
      
            


print ("best hyperparemeters after validation on sketch are : LR={}, NUM_EPOCHS= {} having validation accuracy= {} and validation loss ={}".format(best_LR2, best_epoch_num2,max_val_accuracy2,min_val_loss2))     



hyperprameters are: LR=0.005 and NUM_EPOCHS = 10

Epoch 0001 / 0010


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1/16] class_loss: 2.1138 s_domain_loss: 0.7101 t_domain_loss: 0.7357 
[2/16] class_loss: 1.3060 s_domain_loss: 0.7085 t_domain_loss: 0.7089 
[3/16] class_loss: 0.9358 s_domain_loss: 0.6858 t_domain_loss: 0.7405 
[4/16] class_loss: 0.6901 s_domain_loss: 0.7407 t_domain_loss: 0.6688 
[5/16] class_loss: 0.3901 s_domain_loss: 0.6853 t_domain_loss: 0.7150 
[6/16] class_loss: 0.2917 s_domain_loss: 0.6859 t_domain_loss: 0.7034 
[7/16] class_loss: 0.2664 s_domain_loss: 0.6893 t_domain_loss: 0.6905 
[8/16] class_loss: 0.2520 s_domain_loss: 0.6597 t_domain_loss: 0.7116 
[9/16] class_loss: 0.2211 s_domain_loss: 0.6823 t_domain_loss: 0.6791 
[10/16] class_loss: 0.1428 s_domain_loss: 0.6787 t_domain_loss: 0.6930 
[11/16] class_loss: 0.2312 s_domain_loss: 0.6372 t_domain_loss: 0.7145 
[12/16] class_loss: 0.1514 s_domain_loss: 0.7411 t_domain_loss: 0.5900 
[13/16] class_loss: 0.2153 s_domain_loss: 0.5995 t_domain_loss: 0.7195 
[14/16] class_loss: 0.1353 s_domain_loss: 0.6633 t_domain_loss: 0.6514 
[

  0%|          | 0/16 [00:00<?, ?it/s]

[16/16] class_loss: 0.0901 s_domain_loss: 0.0670 t_domain_loss: 0.0238 


...

Validation Accuracy on Sketch: 0.376953125
hyperprameters are: LR=0.007 and NUM_EPOCHS = 30

Epoch 0001 / 0030
[1/16] class_loss: 2.3260 s_domain_loss: 0.8916 t_domain_loss: 0.5824 
[2/16] class_loss: 1.1057 s_domain_loss: 0.3133 t_domain_loss: 1.3765 
[3/16] class_loss: 0.7840 s_domain_loss: 2.1627 t_domain_loss: 0.1360 
[4/16] class_loss: 0.5933 s_domain_loss: 0.1196 t_domain_loss: 2.3070 
[5/16] class_loss: 0.2917 s_domain_loss: 1.4425 t_domain_loss: 0.2846 
[6/16] class_loss: 0.2315 s_domain_loss: 0.4849 t_domain_loss: 0.9644 
[7/16] class_loss: 0.1761 s_domain_loss: 0.7066 t_domain_loss: 0.6628 
[8/16] class_loss: 0.2437 s_domain_loss: 0.8381 t_domain_loss: 0.5602 
[9/16] class_loss: 0.2218 s_domain_loss: 0.3480 t_domain_loss: 1.1744 
[10/16] class_loss: 0.1879 s_domain_loss: 1.7676 t_domain_loss: 0.1893 
[11/16] class_loss: 0.2463 s_domain_loss: 0.1120 t_domain_loss: 2.2282 
[12/16] class_loss: 0.1818 s_domain_loss: 1.6365 t_domain_loss: 0.2020 
[13/16] class_loss: 0.2158 s_domai

  0%|          | 0/16 [00:00<?, ?it/s]

[16/16] class_loss: nan s_domain_loss: nan t_domain_loss: nan 


100%|██████████| 16/16 [00:15<00:00,  1.03it/s]

Validation Accuracy on Sketch: 0.376953125
best hyperparemeters after validation on sketch are : LR=0.005, NUM_EPOCHS= 10 having validation accuracy= 0.5185546875 and validation loss =10.196216583251953


##Evaluate the best hyperparameters among the two validation sets 

In [0]:
#I take the best hyperparameters belonging to the validation set where the loss is smaller

if min_val_loss<min_val_loss2:
  best_final_LR= best_LR
  best_final_epoch_num= best_epoch_num
else:
    best_final_LR= best_LR2
    best_final_epoch_num= best_epoch_num2

print("best final LR= {}, best final epoch number={}".format(best_final_LR,best_final_epoch_num))

best final LR= 0.007, best final epoch number=20


##Train on Photo and test on Art Paintings with the best hyperparameters with domain adaptation

###Train on photo

In [0]:
alpha=1

net = DANNAlexnet(num_classes = 7)
net = net.to(DEVICE)

class_loss = nn.CrossEntropyLoss() 
domain_loss = nn.CrossEntropyLoss()

parameters_to_optimize = net.parameters() 

optimizer = optim.SGD(parameters_to_optimize, lr=best_final_LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)


max_batches = max(len(train_dataloader), len(test_dataloader))
min_batches = min(len(train_dataloader), len(test_dataloader))#
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda

cudnn.benchmark #function to optimize runtime by finding the best configuration for your hw, useful when input size 
running_corrects = 0
current_step = 0
for epoch in range(best_final_epoch_num):
  print(f'\nEpoch {epoch+1:04d} / {best_final_epoch_num:04d}', end='\n=================\n')
  scheduler.step() 
  train_iterable = iter(train_dataloader)
  test_iterable = iter(test_dataloader)
  for batch in range(max_batches):
    net.train() # Sets module in training mode
    optimizer.zero_grad() # Zero-ing the gradients 
    if( batch == min_batches):
      train_iterable = iter(train_dataloader)#

    images_source, labels_source = next(train_iterable)
    labels_domain = torch.zeros(len(images_source), dtype=torch.long)    
    
    images_source = images_source.to(DEVICE)
    labels_source = labels_source.to(DEVICE)
    labels_domain = labels_domain.to(DEVICE)
    
    class_output = net(images_source)
    domain_output = net(images_source, alpha)
    


    loss_s_label = class_loss(class_output, labels_source)
    loss_s_domain = domain_loss(domain_output, labels_domain)
    

  
    targets, _ = next(test_iterable)
    target_domain = torch.ones(len(targets), dtype=torch.long)

 
    targets = targets.to(DEVICE)
    target_domain = target_domain.to(DEVICE)

    target_output = net(targets, alpha)


    loss_t_domain = domain_loss(target_output,target_domain)


    loss = loss_s_label + loss_s_domain + loss_t_domain
    loss.backward() 

    optimizer.step() 

    current_step += 1

    print(f'[{batch+1}/{max_batches}] '
          f'class_loss: {loss_s_label.item():.4f} ' f's_domain_loss: {loss_s_domain.item():.4f} '
          f't_domain_loss: {loss_t_domain.item():.4f} '
          )  






Epoch 0001 / 0020


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1/8] class_loss: 1.8695 s_domain_loss: 0.5458 t_domain_loss: 0.9141 
[2/8] class_loss: 1.0663 s_domain_loss: 1.4367 t_domain_loss: 0.2952 
[3/8] class_loss: 0.7143 s_domain_loss: 0.1133 t_domain_loss: 2.2845 
[4/8] class_loss: 0.3813 s_domain_loss: 2.0741 t_domain_loss: 0.1473 
[5/8] class_loss: 0.2924 s_domain_loss: 0.2818 t_domain_loss: 1.4189 
[6/8] class_loss: 0.2640 s_domain_loss: 1.1076 t_domain_loss: 0.4096 
[7/8] class_loss: 0.2372 s_domain_loss: 0.3712 t_domain_loss: 1.1456 
[8/8] class_loss: 0.2636 s_domain_loss: 1.4310 t_domain_loss: 0.2752 

Epoch 0002 / 0020
[1/8] class_loss: 0.2075 s_domain_loss: 0.1312 t_domain_loss: 1.9738 
[2/8] class_loss: 0.2272 s_domain_loss: 2.0298 t_domain_loss: 0.1373 
[3/8] class_loss: 0.1358 s_domain_loss: 0.2061 t_domain_loss: 1.5749 
[4/8] class_loss: 0.2274 s_domain_loss: 1.2667 t_domain_loss: 0.3087 
[5/8] class_loss: 0.2107 s_domain_loss: 0.3104 t_domain_loss: 1.2160 
[6/8] class_loss: 0.1241 s_domain_loss: 1.4492 t_domain_loss: 0.2346 
[

###Test on Art Paintings

In [0]:
net = net.to(DEVICE) # this will bring the network to GPU if DEVICE is cuda
net.train(False) # Set Network to evaluation mode

running_corrects = 0
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  # Forward Pass
  outputs = net(images)

  loss = criterion(outputs, labels)

  # Get predictions
  _, preds = torch.max(outputs.data, 1)

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}'.format(accuracy))
print('Loss is: ' + str(loss.item()))

100%|██████████| 8/8 [00:10<00:00,  1.31s/it]

Test Accuracy: 0.4130859375
Loss is: 10.455820083618164
